In [1]:
# !conda install 

In [1]:
import os
import glob
import shutil
from pathlib2 import Path
import platform

# Multi-threading
import multiprocessing
import subprocess

# Web
import urllib.request

In [2]:
# Verify required dependancies including ModisTools are installed properly

if platform.system()=="Windows":
    CP_PROJ_PARAM = "cp_proj_param.exe"
    UNPACK_SDS_BITS = "unpack_sds_bits.exe"
    SUBSET_SDS = "subset_sds.exe"
elif platform.system()=="Linux":
    CP_PROJ_PARAM = "cp_proj_param"
    UNPACK_SDS_BITS = "unpack_sds_bits"
    SUBSET_SDS = "subset_sds"
elif platform.system() == "Darwin":
    CP_PROJ_PARAM = "cp_proj_param"
    UNPACK_SDS_BITS = "unpack_sds_bits"
    SUBSET_SDS = "subset_sds"

In [3]:
print(platform.system())
print(SUBSET_SDS)
print(CP_PROJ_PARAM)
print(UNPACK_SDS_BITS)

os.system("%s --version" % (UNPACK_SDS_BITS))

Linux
subset_sds
cp_proj_param
unpack_sds_bits


0

In [4]:
LDOPE_BIN = "/Users/arawlins/LDOPE-1.7/bin"

In [5]:
inputs = "/media/arawlins/Data/UOM/lfmc-pipeline/NolanLive/modis/"

In [6]:
output = "/media/arawlins/Data/UOM/lfmc-pipeline/NolanLive/derived/"

In [7]:
os.chdir(inputs)

In [8]:
# Setup the search parameters

# DEBUG CODE
product="MOD09A1"
version="6"
# AOI bounding box lower left longitude, lower left latitude, upper right longitude, upper right latitude.
bbox="108.0000,-45.0000,155.0000,-10.0000"
date="2015-01-01,2015-01-05"


# Temporal
# to align with other models use January 1st 2008 to present day.

# Spatial
# Use entire GDA94 bounds (EPSG::3112)


In [10]:

# Query the USGS for Granules and collect list
# Reduce that list to just the granule IDs that align with our Search parameters
# Batch Parallel Download those HDF files

In [9]:
import requests

# Use inventory service to select files
url_string = "https://lpdaacsvc.cr.usgs.gov/services/inventory?product=" + product + "&version=" + \
             version + "&bbox=" + bbox + "&date=" + date + "&output=text"

print(url_string)

r = requests.get(url_string)
print(r.status_code)
print(r.headers['content-type'])
print(r.encoding)

granules = r.text.split('\n')

# These HV values match the SIN projection that covers Australia (EPSG:3112 or GDA94)
acceptable_granules = [
                        [28,11],
                        [28,12],
                        [28,13],
                        [27,12],
                        [29,12],
                        [29,13],
                        [30,10],
                        [30,11],
                        [30,12],
                        [31,10],
                        [31,11],
                        [31,12]
                      ]


    
def get_hv(granule):
    uri_parts = granule.split('/')
    file_name = uri_parts[-1]
    naming_convention_parts = file_name.split('.')
    hv_part = naming_convention_parts[2]
    hv_parts = hv_part.split('v')
    h = hv_parts[0].replace('h', '')
    v = hv_parts[1]
    return [int(h), int(v)]
    
def is_matching_granule(granule):
    return (get_hv(granule) in acceptable_granules)

def download(url):
    """ Requires a vaild .netrc file in users home directory! """
    file_name = url.split('/')[-1]
    
    xml_name = file_name+'.xml'
    
    hdf_file = Path(file_name)
    xml_file = Path(xml_name)
    
    if not hdf_file.is_file():
        print("We will download: " + file_name)
        os.system("curl -n -L -c cookiefile -b cookiefile %s --output %s" % (url, file_name))
    else:
        print(file_name + " exists.")
        
    if not xml_file.is_file():
        os.system("curl -n -L -c cookiefile -b cookiefile %s --output %s" % (url+'.xml', xml_name))
    else:
        print(xml_name + " exists.")
        
    print('Done.')
            

for line in granules:
    if len(line)>0 and is_matching_granule(line):
        download(line)

print("Complete.")

https://lpdaacsvc.cr.usgs.gov/services/inventory?product=MOD09A1&version=6&bbox=108.0000,-45.0000,155.0000,-10.0000&date=2015-01-01,2015-01-05&output=text
200
text/plain; charset=utf-8
utf-8
MOD09A1.A2014361.h27v12.006.2015295040033.hdf exists.
MOD09A1.A2014361.h27v12.006.2015295040033.hdf.xml exists.
Done.
MOD09A1.A2014361.h28v11.006.2015295035818.hdf exists.
MOD09A1.A2014361.h28v11.006.2015295035818.hdf.xml exists.
Done.
MOD09A1.A2014361.h28v13.006.2015295035747.hdf exists.
MOD09A1.A2014361.h28v13.006.2015295035747.hdf.xml exists.
Done.
MOD09A1.A2014361.h31v12.006.2015295040753.hdf exists.
MOD09A1.A2014361.h31v12.006.2015295040753.hdf.xml exists.
Done.
MOD09A1.A2014361.h29v13.006.2015295041049.hdf exists.
MOD09A1.A2014361.h29v13.006.2015295041049.hdf.xml exists.
Done.
MOD09A1.A2014361.h29v12.006.2015295035748.hdf exists.
MOD09A1.A2014361.h29v12.006.2015295035748.hdf.xml exists.
Done.
MOD09A1.A2014361.h30v10.006.2015295040329.hdf exists.
MOD09A1.A2014361.h30v10.006.2015295040329.hdf.x

In [10]:
list_of_hdf = glob.glob("*.hdf")

In [11]:
print(list_of_hdf)

['MOD09A1.A2014361.h29v13.006.2015295041049.hdf', 'MOD09A1.A2014361.h31v12.006.2015295040753.hdf', 'MOD09A1.A2015001.h30v10.006.2015295091243.hdf', 'MOD09A1.A2014361.h30v10.006.2015295040329.hdf', 'MOD09A1.A2014361.h30v11.006.2015295040718.hdf', 'MOD09A1.A2014361.h30v12.006.2015295040403.hdf', 'MOD09A1.A2014361.h31v10.006.2015295040439.hdf', 'MOD09A1.A2014361.h31v11.006.2015295042814.hdf', 'MOD09A1.A2015001.h27v12.006.2015295093248.hdf', 'MOD09A1.A2015001.h28v11.006.2015295091903.hdf', 'MOD09A1.A2015001.h28v12.006.2015295095045.hdf', 'MOD09A1.A2015001.h28v13.006.2015295094941.hdf', 'MOD09A1.A2015001.h29v12.006.2015295092701.hdf', 'MOD09A1.A2015001.h29v13.006.2015295092412.hdf', 'MOD09A1.A2015001.h30v11.006.2015295092707.hdf', 'MOD09A1.A2015001.h30v12.006.2015295092610.hdf', 'MOD09A1.A2015001.h31v10.006.2015295092448.hdf', 'MOD09A1.A2015001.h31v11.006.2015295091835.hdf', 'MOD09A1.A2015001.h31v12.006.2015295092036.hdf', 'MOD09A1.A2014361.h27v12.006.2015295040033.hdf', 'MOD09A1.A2014361.h

In [18]:
os.system("read_sds_attributes %s" % "MOD09A1.A2014361.h29v13.006.2015295041049.hdf")

256

In [15]:
def construct_file_name(prefix, b, name, suffix):
    f_name = prefix + str(b) +'_' + name + '.' + suffix
    return f_name

def quality_control(bit_range, in_file, out_file):
    l_bit = bit_range[0]
    u_bit = bit_range[1]
    os.system("%s -sds=sur_refl_qc_500m -bit=%s-%s -of=%s %s" % (UNPACK_SDS_BITS, l_bit, u_bit, out_file, in_file))
#     print("%s -sds=sur_refl_qc_500m -bit=%s-%s -of=%s %s" % (UNPACK_SDS_BITS, l_bit, u_bit, out_file, in_file))
    
def cloud_cover(lb, ub, prefix, b, in_file):
    o = construct_file_name(prefix, b, in_file[9:16], "h5")
    part_1 = o + '.prj'
    part_2 = construct_file_name("PR_" + prefix, b, in_file[9:16], "h5")
    
    cmd1 = "%s -sds=sur_refl_state_500m -bit=%s-%s -of=%s %s" % (UNPACK_SDS_BITS, lb, ub, o, in_file)
    print(cmd1)
    os.system(cmd1)
    
    cmd2 = "%s -of=%s -ref=%s %s" % (CP_PROJ_PARAM,  part_1, in_file, part_2)
    print(cmd2)
    os.system(cmd2)
#     print("%s -sds=sur_refl_state_500m -bit=%s-%s -of=%s %s" % (UNPACK_SDS_BITS, lb, ub, o, in_file))
#     print("%s %s -ref=%s -of=%s" % (CP_PROJ_PARAM, in_file, part_1, part_2))

In [16]:
# Nolan Live Fuel Model uses...
bands = [1,3,4]

for in_file in list_of_hdf:
    print("Extracting the bands from: " + in_file)
    for b in bands:
        out_file_band = construct_file_name("QCb", b, in_file[9:16], "h5")
        l = ((b-1)*3)+1
        u = l + 3
        bounds = [l,u]
        quality_control(bounds, in_file, out_file_band)
        
        output_band_proj = out_file_band + '.prj'
        output_band_file = construct_file_name("PR_QCb", b, in_file[9:16], "h5")
        os.system("%s -of=%s -ref=%s %s" % (CP_PROJ_PARAM, output_band_proj, in_file, output_band_file))
#         print("%s -of=%s -ref=%s %s" % (CP_PROJ_PARAM, output_band_proj, in_file, output_band_file))
        
        # sur_refl_state_500m bits:
        #         0-1 (cloud state),
        #         2   (cloud shadow),
        #         8-9 (cirrus detected)
        
        # CLoud STate...
        cloud_cover(0, 1, "clst", b, in_file)
        
        # CLoud SHadows...
        cloud_cover(2, 2, "clsh", b, in_file)
        
        # CIrrus DEtected...
        cloud_cover(8, 9, "cide", b, in_file)

print("Complete.")


Extracting the bands from: MOD09A1.A2014361.h29v13.006.2015295041049.hdf
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst1_2014361.h5 MOD09A1.A2014361.h29v13.006.2015295041049.hdf
cp_proj_param -of=clst1_2014361.h5.prj -ref=MOD09A1.A2014361.h29v13.006.2015295041049.hdf PR_clst1_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh1_2014361.h5 MOD09A1.A2014361.h29v13.006.2015295041049.hdf
cp_proj_param -of=clsh1_2014361.h5.prj -ref=MOD09A1.A2014361.h29v13.006.2015295041049.hdf PR_clsh1_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide1_2014361.h5 MOD09A1.A2014361.h29v13.006.2015295041049.hdf
cp_proj_param -of=cide1_2014361.h5.prj -ref=MOD09A1.A2014361.h29v13.006.2015295041049.hdf PR_cide1_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst3_2014361.h5 MOD09A1.A2014361.h29v13.006.2015295041049.hdf
cp_proj_param -of=clst3_2014361.h5.prj -ref=MOD09A1.A2014361.h29v13.006.2015295041049.hdf PR_clst3_2014361.h5
unpack_sds_bits -sd

Extracting the bands from: MOD09A1.A2014361.h30v11.006.2015295040718.hdf
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst1_2014361.h5 MOD09A1.A2014361.h30v11.006.2015295040718.hdf
cp_proj_param -of=clst1_2014361.h5.prj -ref=MOD09A1.A2014361.h30v11.006.2015295040718.hdf PR_clst1_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh1_2014361.h5 MOD09A1.A2014361.h30v11.006.2015295040718.hdf
cp_proj_param -of=clsh1_2014361.h5.prj -ref=MOD09A1.A2014361.h30v11.006.2015295040718.hdf PR_clsh1_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide1_2014361.h5 MOD09A1.A2014361.h30v11.006.2015295040718.hdf
cp_proj_param -of=cide1_2014361.h5.prj -ref=MOD09A1.A2014361.h30v11.006.2015295040718.hdf PR_cide1_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst3_2014361.h5 MOD09A1.A2014361.h30v11.006.2015295040718.hdf
cp_proj_param -of=clst3_2014361.h5.prj -ref=MOD09A1.A2014361.h30v11.006.2015295040718.hdf PR_clst3_2014361.h5
unpack_sds_bits -sd

unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst3_2015001.h5 MOD09A1.A2015001.h27v12.006.2015295093248.hdf
cp_proj_param -of=clst3_2015001.h5.prj -ref=MOD09A1.A2015001.h27v12.006.2015295093248.hdf PR_clst3_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh3_2015001.h5 MOD09A1.A2015001.h27v12.006.2015295093248.hdf
cp_proj_param -of=clsh3_2015001.h5.prj -ref=MOD09A1.A2015001.h27v12.006.2015295093248.hdf PR_clsh3_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide3_2015001.h5 MOD09A1.A2015001.h27v12.006.2015295093248.hdf
cp_proj_param -of=cide3_2015001.h5.prj -ref=MOD09A1.A2015001.h27v12.006.2015295093248.hdf PR_cide3_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst4_2015001.h5 MOD09A1.A2015001.h27v12.006.2015295093248.hdf
cp_proj_param -of=clst4_2015001.h5.prj -ref=MOD09A1.A2015001.h27v12.006.2015295093248.hdf PR_clst4_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh4_2015001.h5 MOD09A1.A2015001.h27v

cp_proj_param -of=cide3_2015001.h5.prj -ref=MOD09A1.A2015001.h29v12.006.2015295092701.hdf PR_cide3_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst4_2015001.h5 MOD09A1.A2015001.h29v12.006.2015295092701.hdf
cp_proj_param -of=clst4_2015001.h5.prj -ref=MOD09A1.A2015001.h29v12.006.2015295092701.hdf PR_clst4_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh4_2015001.h5 MOD09A1.A2015001.h29v12.006.2015295092701.hdf
cp_proj_param -of=clsh4_2015001.h5.prj -ref=MOD09A1.A2015001.h29v12.006.2015295092701.hdf PR_clsh4_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide4_2015001.h5 MOD09A1.A2015001.h29v12.006.2015295092701.hdf
cp_proj_param -of=cide4_2015001.h5.prj -ref=MOD09A1.A2015001.h29v12.006.2015295092701.hdf PR_cide4_2015001.h5
Extracting the bands from: MOD09A1.A2015001.h29v13.006.2015295092412.hdf
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst1_2015001.h5 MOD09A1.A2015001.h29v13.006.2015295092412.hdf
cp_proj_param -of=c

unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide4_2015001.h5 MOD09A1.A2015001.h31v10.006.2015295092448.hdf
cp_proj_param -of=cide4_2015001.h5.prj -ref=MOD09A1.A2015001.h31v10.006.2015295092448.hdf PR_cide4_2015001.h5
Extracting the bands from: MOD09A1.A2015001.h31v11.006.2015295091835.hdf
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst1_2015001.h5 MOD09A1.A2015001.h31v11.006.2015295091835.hdf
cp_proj_param -of=clst1_2015001.h5.prj -ref=MOD09A1.A2015001.h31v11.006.2015295091835.hdf PR_clst1_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh1_2015001.h5 MOD09A1.A2015001.h31v11.006.2015295091835.hdf
cp_proj_param -of=clsh1_2015001.h5.prj -ref=MOD09A1.A2015001.h31v11.006.2015295091835.hdf PR_clsh1_2015001.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide1_2015001.h5 MOD09A1.A2015001.h31v11.006.2015295091835.hdf
cp_proj_param -of=cide1_2015001.h5.prj -ref=MOD09A1.A2015001.h31v11.006.2015295091835.hdf PR_cide1_2015001.h5
unpack_sds_bits -sd

unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst3_2014361.h5 MOD09A1.A2014361.h28v12.006.2015295035931.hdf
cp_proj_param -of=clst3_2014361.h5.prj -ref=MOD09A1.A2014361.h28v12.006.2015295035931.hdf PR_clst3_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh3_2014361.h5 MOD09A1.A2014361.h28v12.006.2015295035931.hdf
cp_proj_param -of=clsh3_2014361.h5.prj -ref=MOD09A1.A2014361.h28v12.006.2015295035931.hdf PR_clsh3_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=8-9 -of=cide3_2014361.h5 MOD09A1.A2014361.h28v12.006.2015295035931.hdf
cp_proj_param -of=cide3_2014361.h5.prj -ref=MOD09A1.A2014361.h28v12.006.2015295035931.hdf PR_cide3_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=0-1 -of=clst4_2014361.h5 MOD09A1.A2014361.h28v12.006.2015295035931.hdf
cp_proj_param -of=clst4_2014361.h5.prj -ref=MOD09A1.A2014361.h28v12.006.2015295035931.hdf PR_clst4_2014361.h5
unpack_sds_bits -sds=sur_refl_state_500m -bit=2-2 -of=clsh4_2014361.h5 MOD09A1.A2014361.h28v

In [15]:
# Nolan - Live Fuel Model - uses...
# sur_refl_b01,
# sur_refl_b02,
# sur_refl_b03,
# sur_refl_b04,
# sur_refl_b06
# sur_refl_b07
bands = [1,2,3,4,6,7]

for in_file in list_of_hdf:
    print("Processing SR bands from: " + in_file)
    for b in bands: 
        output_srb = construct_file_name("SRb", b, in_file[9:16], "h5")
        os.system("%s -sds=sur_refl_b%s -row=0,2400 -col=0,2400 -of=%s %s" % (SUBSET_SDS, str(b), output_srb, in_file))
        
        output_srb_1 = output_srb +'.prj'
        output_srb_2 = construct_file_name("PR_SRb", b, in_file[9:16], "h5")
        os.system("%s -of=%s -ref=%s %s" % (CP_PROJ_PARAM, output_srb_1, in_file, output_srb_2))
                                           
# LOOP THROUGH ALL FILES 
for hdf in glob.glob("*.h5"):

    # ONLY SELECT FINAL OUTPUTS
    if hdf[0:3] == "PR_":
        # COPY FINAL OUTPUTS TO OUTPUT FOLDER
        shutil.copy(hdf, output)

        # REMOVE FINAL OUTPUTS FROM MODIS LDOPE BIN FOLDER
        os.remove(hdf)
        
print("Complete.")

Processing SR bands from: MOD09A1.A2014361.h29v13.006.2015295041049.hdf
Processing SR bands from: MOD09A1.A2014361.h31v12.006.2015295040753.hdf
Processing SR bands from: MOD09A1.A2015001.h30v10.006.2015295091243.hdf
Processing SR bands from: MOD09A1.A2014361.h27v12.006.2015295040033.hdf
Processing SR bands from: MOD09A1.A2014361.h28v11.006.2015295035818.hdf
Processing SR bands from: MOD09A1.A2014361.h28v12.006.2015295035931.hdf
Processing SR bands from: MOD09A1.A2014361.h28v13.006.2015295035747.hdf
Processing SR bands from: MOD09A1.A2014361.h29v12.006.2015295035748.hdf
Processing SR bands from: MOD09A1.A2014361.h30v10.006.2015295040329.hdf
Processing SR bands from: MOD09A1.A2014361.h30v11.006.2015295040718.hdf
Processing SR bands from: MOD09A1.A2014361.h30v12.006.2015295040403.hdf
Processing SR bands from: MOD09A1.A2014361.h31v10.006.2015295040439.hdf
Processing SR bands from: MOD09A1.A2014361.h31v11.006.2015295042814.hdf
Processing SR bands from: MOD09A1.A2015001.h27v12.006.2015295093

In [17]:
os.chdir(output)

In [18]:
print("Looking in: "+ output)

Looking in: /media/arawlins/Data/UOM/lfmc-pipeline/NolanLive/derived/


In [19]:
SRb1_files = glob.glob("PR_SRb*.h5")

In [20]:
print(SRb1_files)

[]


In [22]:
!conda install -y h5py xarray rasterio

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/anaconda3:

The following NEW packages will be INSTALLED:

    h5netcdf: 0.5.0-py_0           conda-forge
    xarray:   0.10.0-py36_0        conda-forge

The following packages will be UPDATED:

    h5py:     2.7.0-py36he81ebca_1             --> 2.7.1-py36_2 conda-forge

h5py-2.7.1-py3 100% |################################| Time: 0:00:10 403.32 kB/s
h5netcdf-0.5.0 100% |################################| Time: 0:00:00  15.65 MB/s
xarray-0.10.0- 100% |################################| Time: 0:00:02 272.92 kB/s


In [23]:
import h5py
import rasterio
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ImportError: /opt/anaconda3/lib/python3.6/site-packages/rasterio/../../.././libkea.so.1.4: undefined symbol: _ZN2H56H5FileC1ERKSsjRKNS_17FileCreatPropListERKNS_15FileAccPropListE

In [48]:
for in_file in SRb1_files:
    print("Examining: ", output + in_file)
    f=Path(output) / in_file
    with h5py.File(f, 'r') as dataset:
        print(dataset)
        for ds in dataset:
            print(ds, " -> ", dataset[ds])

Examining:  /Users/anthonyrawlins/Documents/UOM/NolanLive/derived/PR_SRb1_2015001.h5


OSError: Unable to open file (File signature not found)

In [ ]:
for in_file in SRb1_files:
    with rasterio.open(in_file) as dataset:
        print(dataset.count)

In [45]:
for in_file in SRb1_files:
    with xr.open_dataset(in_file) as ds:
        for k in ds.attrs.keys():
            print(k, " --> ", type(ds.attrs[k]))

# <xarray.Dataset>
# Dimensions:  ()
# Data variables:
#     *empty*
# Attributes:
#     HDFEOSVersion:      HDFEOS_V2.18
#     StructMetadata.0:   GROUP=SwathStructure\nEND_GROUP=SwathStructure\nGROUP...
#     CoreMetadata.0:     \nGROUP                  = INVENTORYMETADATA\n  GROUP...
#     ArchiveMetadata.0:  \nGROUP                  = ARCHIVEDMETADATA\n  GROUPT...

HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class 'str'>
HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class 'str'>
HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class 'str'>
HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class 'str'>
HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class 'str'>
HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class 'str'>
HDFEOSVersion  -->  <class 'str'>
StructMetadata.0  -->  <class 'str'>
CoreMetadata.0  -->  <class 'str'>
ArchiveMetadata.0  -->  <class